# import

In [1]:
import pandas as pd
from tqdm.notebook import tqdm as tqdm
import requests
from bs4 import BeautifulSoup
import re
import time
import urllib.request
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score,roc_curve, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import average_precision_score
import numpy as np
import matplotlib.pyplot as plt
from graphviz import *
from sklearn.preprocessing import LabelEncoder
import optuna.integration.xgboost as xgb_o
import optuna.integration.lightgbm as lgb_o
import scipy as sp
import lightgbm as lgb
import optuna
import sklearn
from scipy.special import comb
from itertools import combinations
import copy
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from scipy.special import comb
from itertools import permutations
import datetime
import lxml
import seaborn as sns
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
from my_library.horse import *

# path

In [3]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_mac2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_mac = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'

# load data

In [4]:
results = load_csv(path_mac+'results.csv')
horse_results = load_csv(path_mac+'horse_results.csv')
peds = load_csv(path_mac+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_mac+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [37]:
lgbm_params = {
                'lambdarank_truncation_level': 2,
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'learning_rate': 0.03508204271531795,
                'boosting_type': 'gbdt',
                'random_state': 777
            }

In [38]:
ll = LearnLGBM(peds,results,horse_results)
ll.learn_lgb(lgbm_params=lgbm_params)


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.086781 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45938
[LightGBM] [Info] Number of data points in the train set: 85124, number of used features: 172


In [5]:
ll = LearnLGBM(peds,results,horse_results)
ll.get_train_data()
train_query = ll.x_train.groupby(ll.x_train.index).size()
val_x = ll.x_test.iloc[:-5000]
val_y = ll.y_test.iloc[:-5000]
test_query = val_x.groupby(val_x.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
valid = lgb.Dataset(val_x, val_y, reference=train, group=test_query)

In [32]:
def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用
    # train = lgb.Dataset(x_train, y_train, group=train_query)
    # valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

    
    lgb_clf = lgb.train(
       params,
       train,
       num_boost_round=1000,
       valid_sets=valid,
       valid_names=['valid'],
       early_stopping_rounds=20,
       verbose_eval=5,
       evals_result=lgb_results
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
#         なぜか, uniformだと strに変換されてしまう
#         lambda_rank_truncation_levelは int型
#         よって, int以外はstrに勝手に変換されてしまい, エラーとなったのではないか
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',[2,3,4,5,6,7,15,20,25,30,35,40,43,50,55,60]),
#         best paramsの返り値は, choiceだとindexか？
        'n_estimators': hp.choice('n_estimators',[50,60,70,80,90,100,110,120]),
        'boosting_type': 'gbdt',
        'random_state': 777,
    }

    max_evals = 50      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)

In [33]:
trials = Trials()
optimize(trials)

Training start:                                       
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/rince/Library/Python/3.7/lib/python/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[LightGBM] [Warning]                                  
Auto-choosing col-wise multi-threading, the overhead of testing was 0.152153 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info]                                     
Total Bins 45938                                      
[LightGBM] [Info]                                     
Number of data points in the train set: 85124, number of used features: 172
Training until validation scores don't improve for 20 rounds
[5]	valid's ndcg@1: 0.525695	valid's ndcg@2: 0.498866	valid's ndcg@3: 0.49621
[10]	valid's ndcg@1: 0.524853	valid's ndcg@2: 0.496273	valid's ndcg@3: 0.492116
[15]	valid's ndcg@1: 0.535805	valid's ndcg@2: 0.497654	valid's ndcg@3: 0.495024
[20]	valid's ndcg@1: 0.533277	valid's ndcg@2: 0.503274	valid's ndcg@3: 0.498896
[25]	valid's ndcg@1: 0.536647	valid's ndcg@2: 0.503094	valid's ndcg@3: 0.500327
[30]	valid's ndcg@1: 0.543387	valid's ndcg@2: 0.507106	valid's ndcg@3: 0.502977
[35]	valid's ndcg@1: 

# Simulate


In [39]:
sl = RankSimulater(ll.model)
start_time = time.time()
for odds in [1.1,2.0,3.0,4.0,5.0,6.0,7.0]:
    print("odds",odds)
    sl.calc_tansho(ll.r.data_c.iloc[-5000:],return_tables,odds=odds)
    print("time", time.time() - start_time)

odds 1.1
not_bet_count 0
---------------------
単勝
的中率 : 75 / 356
的中% : 21.07 %
収支   : -4610.0 円
的中レース : {'01': 10, '02': 7, '03': 6, '04': 9, '05': 4, '06': 5, '07': 12, '08': 7, '09': 2, '10': 4, '11': 5, '12': 4}
time 11.311758756637573
odds 2.0
not_bet_count 43
---------------------
単勝
的中率 : 55 / 313
的中% : 17.57 %
収支   : -3430.0 円
的中レース : {'01': 10, '02': 7, '03': 6, '04': 9, '05': 4, '06': 5, '07': 12, '08': 7, '09': 2, '10': 4, '11': 5, '12': 4}
time 21.666910886764526
odds 3.0
not_bet_count 108
---------------------
単勝
的中率 : 30 / 248
的中% : 12.10 %
収支   : -3090.0 円
的中レース : {'01': 10, '02': 7, '03': 6, '04': 9, '05': 4, '06': 5, '07': 12, '08': 7, '09': 2, '10': 4, '11': 5, '12': 4}
time 30.582987785339355
odds 4.0
not_bet_count 177
---------------------
単勝
的中率 : 17 / 179
的中% : 9.50 %
収支   : -710.0 円
的中レース : {'01': 10, '02': 7, '03': 6, '04': 9, '05': 4, '06': 5, '07': 12, '08': 7, '09': 2, '10': 4, '11': 5, '12': 4}
time 40.5599319934845
odds 5.0
not_bet_count 219
----------------

In [10]:
sl = RankSimulater(ll.model)
sl.calc_sanrenpuku(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)


race_id 202206030211
---------------------
not_bet_count 0
三連複
的中率 : 19 / 356
的中% : 5.34 %
収支   : -10970.0 円


In [11]:
sl = RankSimulater(ll.model)
sl.calc_sanrentan(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)


race_id 202206030211
---------------------
not_bet_count 0
三連単
的中率 : 5 / 356
的中% : 1.40 %
収支   : -10340.0 円


In [40]:
pt = Predictor(peds,results,horse_results,race_id_list)
pt.learn_lgb(lgbm_params=lgbm_params)


pe finish



hr finish



[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.067750 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45931
[LightGBM] [Info] Number of data points in the train set: 85124, number of used features: 172


# 新潟

In [27]:
base  = '2022040101'
pt.predict(base+'05')


              馬番    scores
202204010105   1 -1.258613
202204010105   4 -1.258613
202204010105   5 -1.258613
202204010105   6 -1.258613
202204010105   7 -1.258613
202204010105   9 -1.258613
202204010105  10 -1.258613
202204010105  11 -1.258613
202204010105  12 -1.258613
202204010105  13 -1.258613
202204010105  14 -1.258613
202204010105  15 -1.258613
202204010105  16 -1.258613
202204010105   2 -1.290535
202204010105   3 -1.290535
202204010105   8 -1.290535


# 東京

In [28]:
base  = '2022050205'
pt.predict(base+'05')


              馬番    scores
202205020505   1 -1.258613
202205020505   2 -1.258613
202205020505   3 -1.258613
202205020505   4 -1.258613
202205020505   5 -1.258613
202205020505   6 -1.258613
202205020505   7 -1.258613
202205020505   8 -1.258613
202205020505   9 -1.258613
202205020505  10 -1.258613
202205020505  11 -1.258613
202205020505  12 -1.258613
202205020505  13 -1.258613


# 中京

In [29]:
base  = '2022070301'
pt.predict(base+'05')


              馬番    scores
202207030105   1 -1.258613
202207030105   2 -1.258613
202207030105   3 -1.258613
202207030105   4 -1.258613
202207030105   5 -1.258613
202207030105   6 -1.258613
202207030105   7 -1.258613
202207030105   8 -1.258613
202207030105   9 -1.258613
202207030105  10 -1.258613
202207030105  11 -1.258613


In [41]:
pt.show_results_today()


-------------------
predict
              馬番    scores
202205020501   1 -1.236328
202205020501   2 -1.236328
202205020501   3 -1.236328
actual
                0                1                 2            3
202205020501   単勝               12              590円          3人気
202205020501   複勝          12 16 8      200円210円700円    2人気4人気8人気
202205020501   枠連              6 8              720円          2人気
202205020501   馬連            12 16            1,570円          6人気
202205020501  ワイド  12 16 8 12 8 16  720円3,470円2,660円  7人気32人気27人気
202205020501   馬単            12 16            2,910円         10人気
202205020501  3連複          8 12 16           22,780円         66人気
202205020501  3連単          12 16 8           77,870円        239人気
-------------------
predict
              馬番    scores
202205020502   1 -1.236328
202205020502   2 -1.236328
202205020502   3 -1.236328
actual
                0                1                 2            3
202205020502   単勝               15              530円 

# 日付に注意

In [5]:
date = '2022/12/31'

# race_id_list

In [11]:


race_id_list = ['2022050205{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022050204{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022070301{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090212{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022040101{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022030106{}'.format(str(i).zfill(2)) for i in range(1,13)]



# Results scraping

In [7]:
# race_id_list = results.index.astype('str')

results = Results.scrape(race_id_list)

results.to_csv(path_mac+'results_new.csv')

# Horse_results scraping

In [8]:
horse_id_list = results['horse_id'].astype(str).unique()
horse_results = HorseResults.scrape(horse_id_list)
# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'
horse_results.to_csv(path_mac+'horse_results_new.csv')

# Peds scraping

In [9]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
pe_2021.peds_re.to_csv(path_mac+'peds_new.csv')

# Return scraping

In [10]:
returns_2021 = Return.scrape(race_id_list)
returns_2021.to_csv(path_mac+'returns_new.csv')

# update

In [11]:
new_results = update_data(load_csv(path_mac+'results.csv'), load_csv(path_mac+'results_new.csv'))
new_horse_results = update_data(load_csv(path_mac+'horse_results.csv'), load_csv(path_mac+'horse_results_new.csv'))
new_peds = update_data(load_csv(path_mac+'peds.csv'), load_csv(path_mac+'peds_new.csv'))
new_return = update_data(load_csv(path_mac+'return.csv'), load_csv(path_mac+'returns_new.csv'))

# save

In [12]:
new_results.to_csv(path_mac2+'results.csv')
new_horse_results.to_csv(path_mac2+'horse_results.csv')
new_peds.to_csv(path_mac2+'peds.csv')
new_return.to_csv(path_mac2+'return.csv')

# 重要度

In [29]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

                        features  importances
4                       horse_id         1692
5                      jockey_id          663
25               賞金_race_type_5R          151
24               着順_race_type_5R           65
52               着順_race_type_9R           58
10                         着順_5R           56
38                         着順_9R           47
32                      賞金_開催_5R           36
80             着順_race_type_allR           35
11                         賞金_5R           22
53               賞金_race_type_9R           21
40                         着差_9R           19
7                             体重           16
6                             年齢           16
12                         着差_5R           11
73            着順_course_len_allR           10
45              着順_course_len_9R           10
18              賞金_course_len_5R            9
60                      賞金_開催_9R            6
120                      peds_26            6
8                           体重変化  

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [24]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価